Quick code to join results from different sweeps in just one csv file

In [17]:
import pandas as pd
import numpy as np
import glob, os
import ast

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)


In [18]:
# Path containing the csv for each sweep done
list_path =['/Users/wola/Documents/MSS/Personales/GitRepos/PFM_EconomicNewsImpact/data/models_results']

# We concatenate the results of each sweep in a unique dataframe
all_files = [glob.glob(os.path.join(path, 'sweeps_*.csv')) for path in list_path]             
df_from_each_file = (pd.read_csv(file) for list_files in all_files for file in list_files )
df = pd.concat(df_from_each_file, ignore_index=True)


In [20]:
df.head(3)

,Unnamed: 0,model_type,model,direction,sweeps_market_variables,sweep_news_agg,sweep_buy_sell,before_data,sweep_grid,best_score,best_params,f1_microavg,precision_weighted,precision_below_TH,recall_below_TH,precision_above_TH,recall_above_TH,support_below_TH,support_above_TH,report,elapsed_time
0,0,clf,kn,max,basic,ALL_NO_1_1_1,0_120,included,basic,0.75,{'n_neighbors': 51},0.73,0.71,0.74,0.99,0.65,0.07,935.0,357.0,"{'0': {'precision': 0.7356230031948882, 'recal...",3.78
1,1,clf,svc-rbf,max,basic,ALL_NO_1_1_1,0_120,included,basic,0.75,"{'C': 1, 'gamma': 1}",0.72,0.52,0.72,1.00,0.00,0.00,935.0,357.0,"{'0': {'precision': 0.7236842105263158, 'recal...",8.44
2,2,clf,dtree,max,basic,ALL_NO_1_1_1,0_120,included,basic,0.75,"{'max_depth': 4, 'min_samples_leaf': 50}",0.74,0.71,0.74,0.97,0.62,0.11,935.0,357.0,"{'0': {'precision': 0.74185667752443, 'recall'...",0.56


In [21]:
len(df)

3248

In [22]:
df.to_csv('/Users/wola/Documents/MSS/Personales/GitRepos/PFM_EconomicNewsImpact/data/models_results/all_sweeps.csv')

In [7]:
df_agg = pd.read_csv('./models_out/ALL_YES_3_2_1-basic-included-60_120-basic.csv')

FileNotFoundError: File b'./models_out/ALL_YES_3_2_1-basic-included-60_120-basic.csv' does not exist

In [ ]:
df_agg.columns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report


columns_of_model = ['new_id', 'forecast_error_diff_deviation', 'forecast_error_diff_outlier_class', 
                    'previous_error_diff_deviation', 'previous_error_diff_outlier_class', 'fe_accurate', 
                    'fe_better', 'fe_worse', 'pe_accurate', 'pe_better', 'pe_worse', 'High', 'Low', 'Medium',
                    'year', 'week', 'weekday', 'num_news', 'volatility_0_5_after', 'pips_agg_0_5_after', 
                    'volatility_5_10_after', 'pips_agg_5_10_after', 'volatility_10_15_after', 'pips_agg_10_15_after', 
                    'volatility_15_20_after', 'pips_agg_15_20_after', 'volatility_20_25_after', 'pips_agg_20_25_after',
                    'volatility_25_30_after', 'pips_agg_25_30_after', 'volatility_0_30_after', 'pips_agg_0_30_after', 
                    'volatility_0_60_after', 'pips_agg_0_60_after', 'volatility_60_0_before', 'pips_agg_60_0_before']

X = df_agg[columns_of_model].values
y = df_agg['direction_agg_0_120_after'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
clf_dt = DecisionTreeClassifier(min_samples_leaf= 10, max_depth= 3)

In [ ]:
clf_rf = RandomForestClassifier(max_depth= 7, min_samples_leaf= 10, n_estimators= 200)

In [ ]:
clf_vot = VotingClassifier(estimators=[('dt', clf_dt),('rf', clf_rf)])

In [ ]:
clf_vot.fit(X_train, y_train)

In [ ]:
y_predict = clf_vot.predict(X_test)
classification_report(y_test, y_predict, output_dict=True)

Ok, this ensembled model is not better than the two individual ones